# HEATHOTSAUCE.COM SCRAPE

https://heathotsauce.com

In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [3]:
# scrape all 19 pages to get names and urls

names = []
urls = []

# go to each page (print progress)
for i in range(1, 19):
    url = 'https://heathotsauce.com/collections/all?page=' + str(i)
    page = requests.get(url)
    print (i, page)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #in each page, scrape the names
    for j in soup.findAll('span', class_='title product-name-custom'):
        names.append(j.get_text())
    for k in soup.findAll(itemprop = 'url'):
        urls.append(k.get('href'))

1 <Response [200]>
2 <Response [200]>
3 <Response [200]>
4 <Response [200]>
5 <Response [200]>
6 <Response [200]>
7 <Response [200]>
8 <Response [200]>
9 <Response [200]>
10 <Response [200]>
11 <Response [200]>
12 <Response [200]>
13 <Response [200]>
14 <Response [200]>
15 <Response [200]>
16 <Response [200]>
17 <Response [200]>
18 <Response [200]>


In [5]:
urls

['/collections/all/products/1o8-garlic-basil-hot-sauce',
 '/collections/all/products/24-karat-gold',
 '/collections/all/products/357-gold-edition-w-no-9-plutonium',
 '/collections/all/products/357-naga-morich-puree',
 '/collections/all/products/reaper-puree-1',
 '/collections/all/products/3o8-garlic-ginger-hot-sauce',
 '/collections/all/products/4o8-jerk-style-hot-sauce',
 '/collections/all/products/512-pot-sauce',
 '/collections/all/products/5o8-chipotle-hot-sauce',
 '/collections/all/products/6o8-pineapple-habanero-hot-sauce',
 '/collections/all/products/7-pot-panic',
 '/collections/all/products/7o8-7-pot-citrus',
 '/collections/all/products/7o8-optimus-primo-limited-edition',
 '/collections/all/products/8o8-naga-jolokia-hot-sauce',
 '/collections/all/products/8o8-octo-maximus',
 '/collections/all/products/adoboloco-maui-no-ka-oi-trinidad-scorpion',
 '/collections/all/products/adoboloco-pineapple-habanero',
 '/collections/all/products/african-rhino-peri-peri',
 '/collections/all/prod

In [6]:
#combine names and urls into a df
names_urls = pd.DataFrame({'names':names, 'urls':urls})

# add url to start of urls
names_urls['urls'] = names_urls['urls'].apply(lambda x: 'https://heathotsauce.com' + str(x))

In [7]:
names_urls

,names,urls
0,1o8 Garlic Basil,https://heathotsauce.com/collections/all/produ...
1,24 Karat Gold,https://heathotsauce.com/collections/all/produ...
2,357 Gold Edition,https://heathotsauce.com/collections/all/produ...
3,357 Naga Morich Puree,https://heathotsauce.com/collections/all/produ...
4,357 Reaper Puree,https://heathotsauce.com/collections/all/produ...
...,...,...
643,Wuju Original,https://heathotsauce.com/collections/all/produ...
644,Xtra Virgin-Mildly Adulterated,https://heathotsauce.com/collections/all/produ...
645,Xtreme Regret,https://heathotsauce.com/collections/all/produ...
646,XXX Dime Sauce,https://heathotsauce.com/collections/all/produ...


In [8]:
## Go to each unique url, grab name, price, manufacturer, description

## NOTE: I requested too many timesand was timed out. 
## Do this in batches, save as different df, then concat together

name = []
mfc = []
price = []
desc = []
urls = []

for n, url in enumerate(names_urls['urls'][:21]):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
#     name
    try:
        name.append(soup.find('h1', class_='product_name').get_text())
    except:
        name.append('')
#     manufacturer
    try:
        mfc.append(soup.find('p', class_='vendor').get_text())
    except:
        mfc.append('')
#     price
    try:
        price.append(soup.find('span', class_='money').get_text())
    except:
        price.append('')
#     description
    try:
        desc.append(soup.find('div', class_='description').get_text())
    except:
        desc.append('')
#     url
    try:
        urls.append(url)
    except:
        urls.append('')
    
#     print progress, if response != 200, stop kernel. you have been timed out
    print (n, page)

df_1 = pd.DataFrame({'names':name,
                   'mfc':mfc,
                   'price':price,
                   'desc':desc,
                   'url':urls})

0 <Response [200]>
1 <Response [200]>
2 <Response [200]>
3 <Response [200]>
4 <Response [200]>
5 <Response [200]>
6 <Response [200]>
7 <Response [200]>
8 <Response [200]>
9 <Response [200]>
10 <Response [200]>
11 <Response [200]>
12 <Response [200]>
13 <Response [200]>
14 <Response [200]>
15 <Response [200]>
16 <Response [200]>
17 <Response [200]>
18 <Response [200]>
19 <Response [200]>
20 <Response [200]>


In [ ]:
df_1

In [ ]:
# combine all df batches. strip manufacturers of \n
full_df = df_1
full_df['mfc'] = full_df['mfc'].apply(lambda x: x.strip())
full_df['desc'] = full_df['desc'].apply(lambda x: x.strip())

# remove gift sets and empty rows
full_df = full_df[full_df['mfc'] != 'Gift Set']
full_df = full_df[full_df['desc'] != '']
full_df.reset_index(inplace=True, drop=True)

# save df as csv
full_df.to_csv('hot_sauces.csv', encoding='utf-8', index=False)

In [ ]:
full_df